In [1]:
from linkedin.linkedin import (LinkedInAuthentication, LinkedInApplication,
                               PERMISSIONS)


if __name__ == '__main__':
    API_KEY = '81bb87kgey2uen'
    API_SECRET = 'RsrMeXm9RPC86jUR'
    RETURN_URL = 'https://www.linkedin.com/oauth/v2/authorization'
    authentication = LinkedInAuthentication(API_KEY, API_SECRET, RETURN_URL,
                                            PERMISSIONS.enums.values())
    print(authentication.authorization_url)
    application = LinkedInApplication(authentication)

https://www.linkedin.com/uas/oauth2/authorization?scope=r_basicprofile%20rw_nus%20r_network%20r_contactinfo%20w_messages%20rw_groups%20r_emailaddress%20r_fullprofile&state=ff35b8b8243ce7fafbf0d220f28097d3&redirect_uri=https%3A//www.linkedin.com/oauth/v2/authorization&response_type=code&client_id=81bb87kgey2uen


In [6]:
from linkedin import linkedin

API_KEY = "81bb87kgey2uen"
API_SECRET = "RsrMeXm9RPC86jUR"
RETURN_URL = "http://localhost:8000"
authentication = linkedin.LinkedInAuthentication(API_KEY, API_SECRET, RETURN_URL, linkedin.PERMISSIONS.enums.values())
print authentication.authorization_url
application = linkedin.LinkedInApplication(authentication)

https://www.linkedin.com/uas/oauth2/authorization?scope=r_basicprofile%20rw_nus%20r_network%20r_contactinfo%20w_messages%20rw_groups%20r_emailaddress%20r_fullprofile&state=a4ea1f44eec0d7bce2d0376d5a0051b7&redirect_uri=http%3A//localhost%3A8000&response_type=code&client_id=81bb87kgey2uen


In [ ]:

#__author__ = 'Samuel Marks <samuelmarks@gmail.com>'
#__version__ = '0.1.0'

try:
    from urllib.parse import urlparse
except ImportError:
    from urlparse import urlparse
    
#import SimpleHTTPServer 
from SocketServer import ThreadingTCPServer
#from http.server import SimpleHTTPRequestHandler
from SimpleHTTPServer import SimpleHTTPRequestHandler 

from webbrowser import open_new_tab
from json import dumps
from os import environ


from linkedin.linkedin import LinkedInAuthentication, LinkedInApplication, PERMISSIONS

PORT = 8080


class LinkedInWrapper(object):
    """ Simple namespacing """
    API_KEY = environ.get('81bb87kgey2uen')
    API_SECRET = environ.get('RsrMeXm9RPC86jUR')
    RETURN_URL = 'http://localhost:{0}/code'.format(globals()['PORT'])
    authentication = LinkedInAuthentication(API_KEY, API_SECRET, RETURN_URL, PERMISSIONS.enums.values())
    application = LinkedInApplication(authentication)


liw = LinkedInWrapper()
run_already = False
params_to_d = lambda params: {
    l[0]: l[1] for l in map(lambda j: j.split('='), urlparse(params).query.split('&'))
}


class CustomHandler(SimpleHTTPRequestHandler):
    def json_headers(self, status_code=200):
        self.send_response(status_code)
        self.send_header('Content-type', 'application/json')
        self.end_headers()

    def do_GET(self):
        parsedurl = urlparse(self.path)
        authed = liw.authentication.token is not None

        if parsedurl.path == '/code':
            self.json_headers()

            liw.authentication.authorization_code = params_to_d(self.path).get('code')
            self.wfile.write(dumps({'access_token': liw.authentication.get_access_token(),
                                    'routes': list(filter(lambda d: not d.startswith('_'), dir(liw.application)))}).encode('utf8'))
        elif parsedurl.path == '/routes':
            self.json_headers()

            self.wfile.write(dumps({'routes': list(filter(lambda d: not d.startswith('_'), dir(liw.application)))}).encode('utf8'))
        elif not authed:
            self.json_headers()

            if not globals()['run_already']:
                open_new_tab(liw.authentication.authorization_url)
            globals()['run_already'] = True
            self.wfile.write(dumps({'path': self.path, 'authed': type(liw.authentication.token) is None}).encode('utf8'))
        elif authed and len(parsedurl.path) and parsedurl.path[1:] in dir(liw.application):
            self.json_headers()
            self.wfile.write(dumps(getattr(liw.application, parsedurl.path[1:])()).encode('utf8'))
        else:
            self.json_headers(501)
            self.wfile.write(dumps({'error': 'NotImplemented'}).encode('utf8'))


if __name__ == '__main__':
    httpd = ThreadingTCPServer(('localhost', PORT), CustomHandler)

    print('Server started on port:{}'.format(PORT))
    httpd.serve_forever()


Server started on port:8080


127.0.0.1 - - [01/Nov/2018 12:05:53] "GET / HTTP/1.1" 200 -
Traceback (most recent call last):
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/SocketServer.py", line 596, in process_request_thread
    self.finish_request(request, client_address)
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/BaseHTTPServer.py", line 340, in handle
    self.handle_one_request()
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/BaseHTTPServer.py", line 328, in handle_one_request
    method()
  File "<ipython-input-1-eb56236f3672>", line 65, in do_GET
    open_new_tab(liw.authentication.authorization_url)
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/site-packages/linkedin/linkedin.py", line 96, in authoriza

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 38876)
----------------------------------------
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 38880)
----------------------------------------


127.0.0.1 - - [01/Nov/2018 12:15:33] "GET /code/ HTTP/1.1" 200 -
Traceback (most recent call last):
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/SocketServer.py", line 596, in process_request_thread
    self.finish_request(request, client_address)
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/BaseHTTPServer.py", line 340, in handle
    self.handle_one_request()
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/BaseHTTPServer.py", line 328, in handle_one_request
    method()
  File "<ipython-input-1-eb56236f3672>", line 65, in do_GET
    open_new_tab(liw.authentication.authorization_url)
  File "/home/ankita/anaconda3/envs/py27/lib/python2.7/site-packages/linkedin/linkedin.py", line 96, in auth

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 39016)
----------------------------------------
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 39020)
----------------------------------------


In [12]:
! which python

/home/ankita/anaconda3/bin/python


In [2]:
from linkedin import linkedin

# Define CONSUMER_KEY, CONSUMER_SECRET,  
# USER_TOKEN, and USER_SECRET from the credentials 
# provided in your LinkedIn application

# Instantiate the developer authentication class

CONSUMER_KEY = "8149bpotb7qi6y"
CONSUMER_SECRET = "nfcoqQeepGS3kfqP"
RETURN_URL = "http://localhost:8000"

authentication = linkedin.LinkedInDeveloperAuthentication(CONSUMER_KEY, CONSUMER_SECRET, 
                                                          'ac291096a@gmail.com', 'Ankita_29', 
                                                          RETURN_URL, linkedin.PERMISSIONS.enums.values())

# Pass it in to the app...

application = linkedin.LinkedInApplication(authentication)

# Use the app....

application.get_profile()

LinkedInError: 401 Client Error: Unauthorized for url: https://api.linkedin.com/v1/people/~: Unknown Error

In [ ]:
from linkedin import linkedin

# Define CONSUMER_KEY, CONSUMER_SECRET,
# USER_TOKEN, and USER_SECRET from the credentials
# provided in your LinkedIn application

# Instantiate the developer authentication class

authentication = linkedin.LinkedInDeveloperAuthentication(
                    CONSUMER_KEY,
                    CONSUMER_SECRET,
                    USER_TOKEN,
                    USER_SECRET,
                    RETURN_URL,
                    linkedin.PERMISSIONS.enums.values()
                )

# Optionally one can send custom "state" value that will be returned from OAuth server
# It can be used to track your user state or something else (it's up to you)
# Be aware that this value is sent to OAuth server AS IS - make sure to encode or hash it

# authorization.state = 'your_encoded_message'

# Pass it in to the app...

application = linkedin.LinkedInApplication(authentication)

# Use the app....

application.get_profile()